# Machine Learning: Classification

### 3.1 Churn Prediction

* Data from https://www.kaggle.com/blastchar/telco-customer-churn

### 3.2 Data preparation

* Download the data, read it with pandas
* Look at the data
* Make column names and values look uniform
* Check if all the columns read correctly
* Check if the churn variable needs any preparation

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [2]:
 df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
# transpose the DataFrame to see the entire record of a single entity
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [4]:
# make the data uniform by processing inconsistencies

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')


In [5]:
# verify dataset inconsistencies have been remove

df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [6]:
# peruse the data types

df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [7]:
# zooming in on "totalcharges" data types. Speculating it contains strings, numbers, and maybe whitespaces, underscore, etc
df.totalcharges         

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: totalcharges, Length: 7043, dtype: object

In [8]:
# attempt to convert "totalcharges" to numbers throws an error
# pd.to_numeric(df.totalcharges)

# ValueError: Unable to parse string "_" at position 488

In [9]:
# force pandas to replace values it can't parse with number

tc = pd.to_numeric(df.totalcharges, errors='coerce')

In [10]:
# check for missing values in total charges
tc.isnull().sum()


np.int64(11)

In [11]:
# zoom-in on few columns in total charges with missing values

df[tc.isnull()][['customerid', 'totalcharges']]

,customerid,totalcharges
488,4472-lvygi,_
753,3115-czmzd,_
936,5709-lvoeq,_
1082,4367-nuyao,_
1340,1371-dwpaz,_
3331,7644-omvmy,_
3826,3213-vvolg,_
4380,2520-sgtta,_
5218,2923-arzlg,_
6670,4075-wkniu,_


In [12]:
# force pandas to replace values it can't parse with number

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [13]:
# fill the missing values in total charges with zeros

df.totalcharges = df.totalcharges.fillna(0)

In [14]:
# confirm the columns in total charges with missing values are filled

df[tc.isnull()][['customerid', 'totalcharges']]

,customerid,totalcharges
488,4472-lvygi,0.0
753,3115-czmzd,0.0
936,5709-lvoeq,0.0
1082,4367-nuyao,0.0
1340,1371-dwpaz,0.0
3331,7644-omvmy,0.0
3826,3213-vvolg,0.0
4380,2520-sgtta,0.0
5218,2923-arzlg,0.0
6670,4075-wkniu,0.0


In [15]:
# examine churn variables
df.churn

0        no
1        no
2       yes
3        no
4       yes
       ... 
7038     no
7039     no
7040     no
7041    yes
7042     no
Name: churn, Length: 7043, dtype: object

In [16]:
# examine first 5 churn variables, converting yes/no to True/False
(df.churn == 'yes').head()

0    False
1    False
2     True
3    False
4     True
Name: churn, dtype: bool

In [17]:
# convert the True/False churn variables to 1's and 0's
(df.churn == 'yes').astype('int').head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

In [18]:
# write the  1's and 0's churn variables back into churn
df.churn = (df.churn == 'yes').astype('int')

### 3.3 Setting up the validation framework

* Perform the train/validation/test split with Scikit-Learn

In [19]:
# using package :from sklearn.model_selection import train_test_split

# review sklearn train_test documentation
# ascertain what value to specify for "test_size" parameter
train_test_split?


Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [20]:
# specifying 20% for "test_size" parameter
# NOTE: The train_test_split function split data into, training and testing

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [21]:
# peruse the sizes of the dataset: df_full_train, df_test

len(df_full_train), len(df_test)

(5634, 1409)

In [22]:
# NOTE: since df_test size = 20%, df_full_train contains 80%
# df_full_train is further split into train (75% of df_full_train%) and val (25% of df_full_train)

df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [23]:
# confirm the sizes of the dataset: df_train, df_val and df_test
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [24]:
# the data for all 3 datasets are shuffled. Also notice "churn" is still included and index is not sequential.
df_train.head().T

,3897,1980,6302,727,5104
customerid,8015-ihcgw,1960-uycnn,9250-wypll,6786-obwqr,1328-euzhc
gender,female,male,female,female,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,yes,yes
dependents,yes,no,no,yes,no
tenure,72,10,5,5,18
phoneservice,yes,yes,yes,yes,yes
multiplelines,yes,yes,yes,no,no
internetservice,fiber_optic,fiber_optic,fiber_optic,fiber_optic,no
onlinesecurity,yes,no,no,no,no_internet_service


In [25]:
# reset the index for all 3 datasets. This has no impact on the performance.

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [26]:
# confirm data is sequential
df_train

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,...,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15,0
1,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,...,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55,0
2,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65,1
3,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,...,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10,0
4,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4220,1309-xgfsn,male,1,yes,yes,52,yes,yes,dsl,no,...,yes,no,yes,yes,one_year,yes,electronic_check,80.85,4079.55,0
4221,4819-hjpiw,male,0,no,no,18,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,no,mailed_check,25.15,476.80,0
4222,3703-vavcl,male,0,yes,yes,2,yes,no,fiber_optic,no,...,yes,yes,no,yes,month-to-month,no,credit_card_(automatic),90.00,190.05,1
4223,3812-lrzir,female,0,yes,yes,27,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,electronic_check,24.50,761.95,0


In [27]:
# split the churn into datasets

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [28]:
# preview the data

y_train, y_val, y_test

(array([0, 0, 1, ..., 1, 0, 1]),
 array([0, 0, 0, ..., 0, 1, 1]),
 array([0, 0, 0, ..., 0, 0, 1]))

In [29]:
# delete y_train, y_val, and y_test from the datasets to avoid accidentally training the model with them, which would lead to overfitting

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [30]:
# confirm deletion of y_train i.e "churn". Now, notice that "churn" is not included and the dataset is also sequential

df_train.head().T # churn non-inclusion can also be confirmed with df_train.dtypes



,0,1,2,3,4
customerid,8015-ihcgw,1960-uycnn,9250-wypll,6786-obwqr,1328-euzhc
gender,female,male,female,female,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,yes,yes
dependents,yes,no,no,yes,no
tenure,72,10,5,5,18
phoneservice,yes,yes,yes,yes,yes
multiplelines,yes,yes,yes,no,no
internetservice,fiber_optic,fiber_optic,fiber_optic,fiber_optic,no
onlinesecurity,yes,no,no,no,no_internet_service


### 3.4 Exploratory Data Analysis (EDA)

* Check missing values
* Look at the target variable (churn)
* Look at numerical and categorical variables

In [31]:
# reset index for df_full_train

df_full_train = df_full_train.reset_index(drop=True)

In [32]:
# check for missing values in df_full_train. No additional data preparation is necessary as the only missing values found in column "totalcharges" has been managed.
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [33]:
# check for missing values in target variable "df_full_train.churn"
df_full_train.churn

0       0
1       1
2       0
3       0
4       0
       ..
5629    1
5630    0
5631    1
5632    1
5633    0
Name: churn, Length: 5634, dtype: int64

In [34]:
# examine the distribution "df_full_train.churn" to ascertain churning and non-churning users. How many times 1's and 0's occur
df_full_train.churn.value_counts()

churn
0    4113
1    1521
Name: count, dtype: int64

In [35]:
# ascertain the percentage distribution of churning and non-churning users. That is, ascertain the CHURN RATE
df_full_train.churn.value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

$$
\text{Global Churn Rate} = \frac{1}{n} \sum_{i=1}^{n} X_i
$$

In [36]:
# ALTERNATELY, ascertain the CHURN RATE by computing the mean
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

np.float64(0.27)

In [37]:
# examine the categorical and numerical variables
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [38]:
# create a list of numerical variables: tenure, monthlycharges, and totalcharges.

numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [39]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [40]:
# create a list of categorical variable

categorical = ['gender', 'seniorcitizen', 'partner', 'dependents','phoneservice',  'multiplelines', 'internetservice','onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport','streamingtv', 'streamingmovies', 'contract', 'paperlessbilling','paymentmethod']

In [41]:
# view the categorical subset
df_full_train[categorical]

,gender,seniorcitizen,partner,dependents,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod
0,male,0,yes,yes,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check
1,female,0,no,no,yes,no,dsl,yes,yes,yes,yes,no,yes,one_year,no,credit_card_(automatic)
2,male,0,yes,no,yes,yes,dsl,yes,yes,no,yes,no,no,two_year,no,bank_transfer_(automatic)
3,male,0,yes,yes,yes,yes,dsl,yes,no,yes,yes,yes,yes,one_year,no,electronic_check
4,male,0,no,no,yes,no,dsl,yes,yes,no,yes,yes,no,one_year,no,electronic_check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,male,1,no,no,yes,yes,fiber_optic,no,no,yes,no,yes,yes,month-to-month,yes,electronic_check
5630,male,0,no,yes,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check
5631,male,0,no,no,yes,yes,fiber_optic,no,yes,yes,no,yes,yes,month-to-month,yes,electronic_check
5632,male,0,no,no,yes,yes,dsl,no,yes,no,no,no,no,month-to-month,yes,mailed_check


In [42]:
# calculate the unique value of categorical subset
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### 3.5 Feature importance: Churn rate and risk ratio

**Feature importance analysis (part of EDA) - identifying which features affect our target variable**

* Churn rate
* Risk ratio
* Mutual information - Later 



* Churn rate

In [43]:
# examine churn rate within each / different groups
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [44]:
# compute the global churn rate irrespective of variable class (numerical or categorical)
global_churn = df_full_train.churn.mean()
global_churn

np.float64(0.26996805111821087)

In [45]:
# examine churn rate within gender: 
# compute churn rate within female customers
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

np.float64(0.27682403433476394)

In [46]:
# compute churn rate within male customers
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

np.float64(0.2632135306553911)

In [66]:
# less tha 1% difference between global churn rate and churn rates within female customers
diff_global_female = global_churn - churn_female

# INTERPRETATION: global_churn rate < zero implies female customers are MORE LIKELY to churn. But a negative  0.7% churn rate difference isn't so significant.
diff_global_female

np.float64(-0.006855983216553063)

In [65]:
# less than 1% difference between global churn rate and churn rates within male customers
diff_global_male = global_churn - churn_male

# INTERPRETATION: global_churn rate > zero implies male customers are LESS LIKELY to churn. But positive 0.7% churn rate difference isn't so significant.
diff_global_male

np.float64(0.006754520462819769)

In [67]:
# approximately 0.7% (less tha 1%) for female and male: an insignificant difference.

# INTERPRETATION: for gender, their rate is less than 1%, not only intangible but almost the same as any customer in the global customer base and so possesses little or no risk. Overall, gender is less significant a variable for impactful churn prediction outcomes.

# the focus is on how large the differences between the global churn rate and the churn rates of the variable it's compared against are.

diff_global_female, diff_global_male

(np.float64(-0.006855983216553063), np.float64(0.006754520462819769))

In [49]:
# Next, compute the churn rate within other variables: partners
df_full_train.partner.value_counts()

partner
no     2932
yes    2702
Name: count, dtype: int64

In [50]:
# compute churn rate for partner variable: for customers with partner
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

np.float64(0.20503330866025166)

In [51]:
# compute churn rate for partner variable: for customers with no partners
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

np.float64(0.3298090040927694)

In [64]:
# approximate 6% difference between global churn rate and churn rates for customers with partners
diff_global_churn_partner = global_churn - churn_partner

# INTERPRETATION: global_churn rate > zero implies customers with partners are LESS LIKELY to churn. A positive 6% churn rate difference is highly significant.
diff_global_churn_partner

np.float64(0.06493474245795922)

In [63]:
# approximate 6% difference between global churn rate and rate for customers with no partners
diff_global_churn_no_partner = global_churn - churn_no_partner

# INTERPRETATION: global_churn rate < zero implies customers with no partners are MORE LIKELY to churn. A negative 6% churn rate difference is highly significant.
diff_global_churn_no_partner

np.float64(-0.05984095297455855)

* Risk ratio

In [61]:
# ratio churn rate: churn_no_partner / global_churn
ratio_churn_no_partner_to_global = churn_no_partner / global_churn

# INTERPRETATION: with ratio > 1 implies are MORE LIKELY to churn.Their churn rate is relatively 22% higher. This correlates with the strong negative 6% in absolute terms.
ratio_churn_no_partner_to_global

np.float64(1.2216593879412643)

In [60]:
# percentage churn rate: churn_partner / global_churn
ratio_churn_partner_to_global = churn_partner  / global_churn

# INTERPRETATION: with ratio < 1 implies are LESS LIKELY to churn. Their churn rate is relatively 24% lower than the global churn
ratio_churn_partner_to_global

np.float64(0.7594724924338315)

```sql
SELECT
    gender,
    AVG(churn),
    AVG(churn) - global_churn AS diff,
    AVG(churn) / global_churn AS risk

FROM
    data

GROUP BY
    gender;

In [73]:
# compute the average churn by gender
df_full_train.groupby('gender').churn.mean() 

gender
female    0.276824
male      0.263214
Name: churn, dtype: float64

In [75]:
# convert the "average churn by gender" series into a DataFrame
df_full_train.groupby('gender').churn.agg(['mean','count'])

,mean,count
gender,,
female,0.276824,2796
male,0.263214,2838


In [78]:
# add the "average churn by gender" into a group DataFrame
df_group = df_full_train.groupby('gender').churn.agg(['mean','count'])

# add more groups to the DataFrame
df_group['diff'] = df_group['mean'] - global_churn
df_group['risk'] = df_group['mean'] / global_churn
df_group

,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980


In [82]:
# defining a function specific to Jupyter Notebook to display the "df_group" within the for loop
from IPython.display import display

In [85]:
# repeat the above analysis for all categorical variables
for c in categorical:
    print(c) # add the name of the variable being evaluated
    df_group = df_full_train.groupby(c).churn.agg(['mean','count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980



seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208



partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730



paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560



paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


### 3.6 Feature importance: Mutual information

**Mutual information - concept from information theory, it tells us how much we can learn about one variable if we know the value of another**

* https://en.wikipedia.org/wiki/Mutual_information

In [86]:
from sklearn.metrics import mutual_info_score

In [88]:
# indicates contract type provides significant information about chances of churning
mutual_info_score(df_full_train.contract, df_full_train.churn)

np.float64(0.0983203874041556)

In [89]:
# indicates gender provides little or no information about chances of churning
mutual_info_score(df_full_train.gender, df_full_train.churn)

np.float64(0.0001174846211139946)

In [91]:
# indicates partner is relatively more important than gender but less than contract
mutual_info_score(df_full_train.partner, df_full_train.churn)

np.float64(0.009967689095399745)

In [92]:
# apply the technique of "mutual_info_score" analysis to all the categorical variables to ascertain the highest score

# define a function that takes a series
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [93]:
df_full_train[categorical].apply( mutual_info_churn_score)

gender              0.000117
seniorcitizen       0.009410
partner             0.009968
dependents          0.012346
phoneservice        0.000229
multiplelines       0.000857
internetservice     0.055868
onlinesecurity      0.063085
onlinebackup        0.046923
deviceprotection    0.043453
techsupport         0.061032
streamingtv         0.031853
streamingmovies     0.031581
contract            0.098320
paperlessbilling    0.017589
paymentmethod       0.043210
dtype: float64

In [94]:
# sort so the most important variable or variable with the hightest score comes first
mi = df_full_train[categorical].apply( mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

### 3.7 Feature importance: Correlation

**How about numerical columns?**

* Correlation coefficient